# BLIP-2 Medical VQA Training - Abdomen (1 Epoch Validation)

Train BLIP-2 on fetal ultrasound abdomen images Q&A for 1 epoch to validate pipeline.

**Model**: Salesforce/blip2-opt-2.7b (8-bit)
**Data**: 5 images, 40 Q&A pairs, split 3/1/1
**Training time**: ~10 minutes

In [1]:
# Parameters (can be overridden by papermill)
num_images = 5
num_epochs = 1
batch_size = 1
learning_rate = 1e-4

In [2]:
import os
import sys
import torch
import pandas as pd
from pathlib import Path
from PIL import Image
from transformers import Blip2Processor, Blip2ForConditionalGeneration, TrainingArguments, Trainer
from torch.utils.data import Dataset
import time

print(f"CUDA: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")

CUDA: True
GPU: NVIDIA GeForce RTX 4070 Laptop GPU


In [3]:
# Paths
BASE = Path(r"C:\Users\elyas\Workspace\PyCharm\fada-v3")
DATA_DIR = BASE / "data/Fetal Ultrasound Labeled"
IMAGE_DIR = BASE / "data/Fetal Ultrasound/Abodomen"
OUTPUT_DIR = BASE / "outputs/blip2_abdomen"
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

MODEL_NAME = "Salesforce/blip2-opt-2.7b"
print(f"Output: {OUTPUT_DIR}")

Output: C:\Users\elyas\Workspace\PyCharm\fada-v3\outputs\blip2_abdomen


In [4]:
# Load data
df = pd.read_excel(DATA_DIR / "Abodomen_image_list.xlsx")
q_cols = [c for c in df.columns if c.startswith('Q')]

vqa_data = []
for _, row in df.head(num_images).iterrows():
    img_path = IMAGE_DIR / row['Image Name']
    if not img_path.exists():
        continue
    for q_col in q_cols:
        q = q_col.split('\n', 1)[1][:100] if '\n' in q_col else q_col[:100]
        a = str(row[q_col])
        if pd.notna(a) and a.lower() not in ['nan', 'none', '']:
            vqa_data.append({'image_path': str(img_path), 'question': q, 'answer': a})

# Split: 70% train, 15% val, 15% test
n_train = int(len(vqa_data) * 0.7)
n_val = int(len(vqa_data) * 0.15)
train_data = vqa_data[:n_train]
val_data = vqa_data[n_train:n_train+n_val]
test_data = vqa_data[n_train+n_val:]

print(f"Train: {len(train_data)}, Val: {len(val_data)}, Test: {len(test_data)}")

Train: 28, Val: 6, Test: 6


In [5]:
# Load model with LoRA
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training

processor = Blip2Processor.from_pretrained(MODEL_NAME)
model = Blip2ForConditionalGeneration.from_pretrained(
    MODEL_NAME,
    load_in_8bit=True,
    device_map="auto"
)

# Prepare for training
model = prepare_model_for_kbit_training(model)

# Add LoRA adapters
lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.05,
    bias="none"
)
model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

print(f"Memory: {model.get_memory_footprint() / 1e9:.2f} GB")

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

trainable params: 2,621,440 || all params: 3,747,383,296 || trainable%: 0.0700
Memory: 4.48 GB


In [6]:
# Dataset
class VQADataset(Dataset):
    def __init__(self, data, processor):
        self.data = data
        self.processor = processor
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        item = self.data[idx]
        img = Image.open(item['image_path']).convert('RGB')
        prompt = f"Question: {item['question']} Answer:"
        
        inputs = self.processor(
            images=img,
            text=prompt,
            return_tensors="pt",
            padding="max_length",
            max_length=256,
            truncation=True
        )
        
        labels = self.processor.tokenizer(
            item['answer'],
            return_tensors="pt",
            padding="max_length",
            max_length=128,
            truncation=True
        )["input_ids"]
        
        return {
            'pixel_values': inputs['pixel_values'].squeeze(),
            'input_ids': inputs['input_ids'].squeeze(),
            'labels': labels.squeeze()
        }

train_dataset = VQADataset(train_data, processor)
val_dataset = VQADataset(val_data, processor)
print(f"Datasets ready: {len(train_dataset)} train, {len(val_dataset)} val")

Datasets ready: 28 train, 6 val


In [7]:
# Training config
training_args = TrainingArguments(
    output_dir=str(OUTPUT_DIR),
    num_train_epochs=num_epochs,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    learning_rate=learning_rate,
    logging_steps=5,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    report_to="none",
    remove_unused_columns=False
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset
)

print("Trainer ready")

Trainer ready


In [8]:
# Train
start_time = time.time()
train_result = trainer.train()
training_time = time.time() - start_time

print(f"\nTraining complete!")
print(f"Time: {training_time/60:.2f} minutes")
print(f"Final loss: {train_result.training_loss:.4f}")

# Save
trainer.save_model(str(OUTPUT_DIR / "final_model"))
processor.save_pretrained(str(OUTPUT_DIR / "final_model"))
print(f"Model saved to {OUTPUT_DIR / 'final_model'}")

C:\Users\elyas\Workspace\PyCharm\fada-v3\venv\lib\site-packages\torch\_dynamo\eval_frame.py:929: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
C:\Users\elyas\Workspace\PyCharm\fada-v3\venv\lib\site-packages\torch\utils\checkpoint.py:85: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(
C:\Users\elyas\Workspace\PyCharm\fada-v3\venv\lib\site-packages\bitsandbytes\autograd\_functions.py:186: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.


Epoch,Training Loss,Validation Loss
1,4.561800,4.253651



Training complete!
Time: 0.37 minutes
Final loss: 5.4157


Model saved to C:\Users\elyas\Workspace\PyCharm\fada-v3\outputs\blip2_abdomen\final_model


In [9]:
# Test inference
model.eval()
test_item = test_data[0]
test_img = Image.open(test_item['image_path']).convert('RGB')
test_prompt = f"Question: {test_item['question']} Answer:"

inputs = processor(images=test_img, text=test_prompt, return_tensors="pt").to(model.device)

with torch.no_grad():
    outputs = model.generate(**inputs, max_new_tokens=50)
    prediction = processor.batch_decode(outputs, skip_special_tokens=True)[0]

print(f"\nTest Inference:")
print(f"Q: {test_item['question']}")
print(f"Predicted: {prediction}")
print(f"Ground truth: {test_item['answer'][:100]}")

C:\Users\elyas\Workspace\PyCharm\fada-v3\venv\lib\site-packages\bitsandbytes\autograd\_functions.py:186: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")



Test Inference:
Q: Plane Evaluation: Assess if the image is taken at a standard diagnostic plane and describe its diagn
Predicted: Question: Plane Evaluation: Assess if the image is taken at a standard diagnostic plane and describe its diagn Answer: The image is taken at a standard diagnostic plane and describes its diagn

Ground truth: Yes the image is taken at a standard diagnostic plane, but the entire outline of the fetal abdomen i


In [10]:
# Summary
summary = {
    'model': MODEL_NAME,
    'category': 'Abdomen',
    'num_images': num_images,
    'train_samples': len(train_data),
    'val_samples': len(val_data),
    'test_samples': len(test_data),
    'epochs': num_epochs,
    'training_time_min': training_time/60,
    'final_loss': train_result.training_loss,
    'output_dir': str(OUTPUT_DIR)
}

import json
with open(OUTPUT_DIR / 'training_summary.json', 'w') as f:
    json.dump(summary, f, indent=2)

print("\n" + "="*60)
print("TRAINING SUMMARY")
print("="*60)
for k, v in summary.items():
    print(f"{k}: {v}")
print("="*60)


TRAINING SUMMARY
model: Salesforce/blip2-opt-2.7b
category: Abdomen
num_images: 5
train_samples: 28
val_samples: 6
test_samples: 6
epochs: 1
training_time_min: 0.36673975785573326
final_loss: 5.415697165897915
output_dir: C:\Users\elyas\Workspace\PyCharm\fada-v3\outputs\blip2_abdomen
